# SageMaker MLflow with Strands Agents and Amazon Bedrock AgentCore

This tutorial we will introduce Strands Agents and how to deploy it through Amazon Bedrock AgentCore, we will also show you how to use MLflow for observability 

In [ ]:
# Install dependencies for AgentCore deployment. Ignore any warnings and residual dependency errors.
!pip install --force-reinstall -U -r requirements-agentcore.txt --quiet

In [ ]:
# Import deployment tools and utilities
import uuid
from utils import setup_cognito_user_pool, reauthenticate_user, delete_cognito_user_pool
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
from typing import Any, Optional
import urllib.parse
import requests
import json
import sagemaker

# Create a SageMaker session
sess = sagemaker.Session()

# Get the current AWS region
region = sess.boto_region_name

print(f"The current AWS region is: {region}")

## What happens behind the scenes?

To deploy our agents to `AgentCore Runtime`. To do so we need to:

- Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
- Initialize the App in our code with `app = BedrockAgentCoreApp()`
- Decorate the invocation function with the `@app.entrypoint` decorator
- Let AgentCore Runtime control the running of the agent with `app.run()`

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to AWS standards



> In this workshop the IAM permissioning needed for AgentCore runtime deployments are already configured for you. You can learn about the AgentCore runtime IAM permission settings in [AWS documentation](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/runtime-permissions.html).

### Setting up Amazon Cognito for Authentication

AgentCore Runtime requires authentication. We'll use Amazon Cognito to provide JWT tokens for accessing our deployed agent server.

In [ ]:
# Set up Amazon Cognito for AgentCore Runtime authentication
print("Setting up Amazon Cognito user pool...")

cognito_config = setup_cognito_user_pool()

print("Cognito setup completed ✓")
print(f"User Pool ID: {cognito_config.get('user_pool_id', 'N/A')}")
print(f"Client ID: {cognito_config.get('client_id', 'N/A')}")

# Configure JWT authorization for AgentCore Runtime
auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [cognito_config["client_id"]],
        "discoveryUrl": cognito_config["discovery_url"],
    }
}

### Deploying the agent to AgentCore Runtime with SageMaker managed MLflow

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables, and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** From a Operations/DevOps best practice is to package code as a container and push to ECR using CI/CD pipelines.

In this tutorial, we will use the Amazon Bedrock AgentCore Python SDK to package your artifacts and deploy them to AgentCore Runtime.

We will retrieve the stored notebook values containing the SageMakerAI MLflow Tracking Server ARN. If the stored value is empty you can copy the tracking server arn from the SageMakerAI MLflow studio console. You will enter the tracking server arn for the variable `_SAGEMAKER_MLFLOW_URI_ARN`.

In [ ]:
# Retrieve values stored from previous labs
# If the stored value (or if you get NameError) is empty, set your SageMaker Managed MLflow tracking server ARN copied from prerequisites
%store -r 

%store
if TRACKING_SERVER_ARN == "":
    print("ENTER YOUR MLFLOW TRACKING SERVER ARN")
TRACKING_SERVER_ARN


### Configure AgentCore Runtime deployment with SageMaker managed MLflow
We will use `mlflow` python library to enable tracing for the strands agent sdk deployed in AgentCore. To enable tracing we call the `mlflow.autolog()` module which has the integrations needed to automatically process strands agent sdk framework and log the traces into SageMaker managed MLflow. Note: Strands agent framework is support in MLflow version `3.4.0` or higher.

In this workshop lab we will use Amazon Bedrock AgentCore Runtime [starter toolkit](https://github.com/aws/bedrock-agentcore-starter-toolkit) to simplyfy the AgentCore Runtime deployment with an entrypoint. The starter kit will configure the Dockerfile and auto-create the Amazon ECR repository on launch. The starter kit will consume the requirements file containing reference to MLflow `v3.4.0+` to generate the strands agent application code.

During the configure step, your Dockerfile will be generated based on your application code.

![runtime](./static/sagemaker-mlflow-agentCore.png)

Important: In the next notebook cell make sure to set the following variables, 
- `_SAGEMAKER_MLFLOW_URI_ARN`: The ARN of the SageMaker managed MLflow tracking server, for example `arn:aws:sagemaker:<Region>:<AWS-account-id>:mlflow-tracking-server/<Name>`
- `_SAGEMAKER_MLFLOW_EXPERIMENT_NAME`: A name for this MLflow experiment, for example `customer_support_genai_agentcore`
- `_REGION`: The aws region of the SageMaker managed MLflow tracking server and the AgentCore agent, for example `us-east-1`
- `_BEDROCK_MODELID`: The Bedrock model ID, for example `us.anthropic.claude-3-5-haiku-20241022-v1:0`

For the agent use-case we will create a Financial Analysis Agent using [Strands Agents](https://strandsagents.com/latest/)

Financial Analysis Agent: Focuses on investment research, portfolio management, and market analysis. It consists of 3 tools:
- `get_stock_analysis`: Real-time stock data and comprehensive analysis. Example tool use-case "Analyze Apple stock performance and metrics" 
- `create_diversified_portfolio`: Risk-based portfolio recommendations with allocations. Example tool use-case "Create a moderate risk portfolio for $10,000" 
- `compare_stock_performance`: Multi-stock performance comparison over time periods. Example tool use-case "Compare Tesla, Apple, and Google over 6 months" 

<div class="alert alert-block alert-warning">
<b>Important:</b> Update `_SAGEMAKER_MLFLOW_URI_ARN` with your SageMaker Managed MLflow tracking server ARN.
</div>

In [ ]:
%%writefile strands_agentcore_SageMaker_deploy.py
# Create AgentCore-SageMaker compatible deployment file with streaming endpoint

import yfinance as yf
from strands import Agent, tool
from strands.models import BedrockModel


from strands.agent.conversation_manager import SummarizingConversationManager
from bedrock_agentcore import BedrockAgentCoreApp

import mlflow
from typing import List

# Enter the ARN of the SageMaker managed MLflow tracking server
_SAGEMAKER_MLFLOW_URI_ARN =  "ENTER YOUR MLFLOW TRACKING SERVER ARN HERE" 
# Enter a name for this MLflow experiment
_SAGEMAKER_MLFLOW_EXPERIMENT_NAME = "customer_support_genai_agentcore" # <ENTER/REPLACE-VALUE>
# Enter the aws region of the SageMaker managed MLflow tracking server and the AgentCore agent
_REGION = "us-east-1" # <ENTER/REPLACE-VALUE>
# Enter the Bedrock model ID 
_BEDROCK_MODELID = "global.anthropic.claude-haiku-4-5-20251001-v1:0" # <ENTER/REPLACE-VALUE>

# Set the SageMaker managed MLflow ARN
mlflow.set_tracking_uri(_SAGEMAKER_MLFLOW_URI_ARN)
# Set the name for this MLflow experiment
mlflow.set_experiment(_SAGEMAKER_MLFLOW_EXPERIMENT_NAME)

# enable automatic logging of traces
mlflow.autolog()

app = BedrockAgentCoreApp()
agent = Agent()

# Financial Analysis Agent System Prompt
FINANCIAL_ANALYSIS_PROMPT = """You are a specialized financial analysis agent focused on investment research and portfolio recommendations. Your role is to:

1. Research and analyze stock performance data
2. Create diversified investment portfolios
3. Provide data-driven investment recommendations

You do not provide specific investment advice but rather present analytical data to help users make informed decisions. Always include disclaimers about market risks and the importance of consulting financial advisors."""

# Add conversation management to maintain context
conversation_manager = SummarizingConversationManager(
    summary_ratio=0.3,  # Summarize 30% of messages when context reduction is needed
    preserve_recent_messages=5,  # Always keep 5 most recent messages
)

bedrock_model = BedrockModel(
    model_id=_BEDROCK_MODELID,
    region_name=_REGION,
    temperature=0.0,  # Deterministic responses for financial advice
)


# Tool 1: Get Stock Analysis
@tool
def get_stock_analysis(symbol: str) -> str:
    """Get comprehensive analysis for a specific stock symbol. Fetches the current stock price for a given symbol.

    Args:
        symbol: The name of the stock symbol (e.g., 'AMZN', 'AAPL', 'NVDA').

    Returns:
        str: The detailed stock analysis.

    Raises:
        KeyError: If the specified stock is not found in the data source.
    """
    try:
        stock = yf.Ticker(symbol)
        info = stock.info
        hist = stock.history(period="1y")

        # Calculate key metrics
        current_price = hist["Close"].iloc[-1]
        year_high = hist["High"].max()
        year_low = hist["Low"].min()
        avg_volume = hist["Volume"].mean()
        price_change = (
            (current_price - hist["Close"].iloc[0]) / hist["Close"].iloc[0]
        ) * 100

        return f"""
📊 Stock Analysis for {symbol.upper()}:
• Current Price: ${current_price:.2f}
• 52-Week High: ${year_high:.2f}
• 52-Week Low: ${year_low:.2f}
• Year-to-Date Change: {price_change:.2f}%
• Average Daily Volume: {avg_volume:,.0f} shares
• Company: {info.get("longName", "N/A")}
• Sector: {info.get("sector", "N/A")}
"""
    except Exception as e:
        return f"❌ Unable to retrieve data for {symbol}: {str(e)}"


# Tool 2: Create Diversified Portfolio
@tool
def create_diversified_portfolio(risk_level: str, investment_amount: float) -> str:
    """Create a diversified portfolio based on risk level (conservative, moderate, aggressive) and investment amount."""

    portfolios = {
        "conservative": {
            "stocks": ["AAPL", "MSFT", "JNJ", "PG", "KO"],
            "weights": [0.25, 0.25, 0.20, 0.15, 0.15],
            "description": "Focus on large-cap, dividend-paying stocks",
        },
        "moderate": {
            "stocks": ["AAPL", "GOOGL", "AMZN", "TSLA", "NVDA"],
            "weights": [0.30, 0.25, 0.20, 0.15, 0.10],
            "description": "Balanced mix of growth and stability",
        },
        "aggressive": {
            "stocks": ["TSLA", "NVDA", "AMZN", "GOOGL", "META"],
            "weights": [0.30, 0.25, 0.20, 0.15, 0.10],
            "description": "High-growth potential stocks",
        },
    }

    if risk_level.lower() not in portfolios:
        return "❌ Risk level must be: conservative, moderate, or aggressive"

    portfolio = portfolios[risk_level.lower()]

    result = f"""
🎯 {risk_level.upper()} Portfolio Recommendation (${investment_amount:,.0f}):
{portfolio["description"]}

Portfolio Allocation:
"""

    for stock, weight in zip(portfolio["stocks"], portfolio["weights"]):
        allocation = investment_amount * weight
        result += f"• {stock}: {weight * 100:.0f}% (${allocation:,.0f})\n"

    result += "\n⚠️ Disclaimer: This is for educational purposes only. Consult a financial advisor before investing."
    return result


# Tool 3: Compare Stock Performance
@tool
def compare_stock_performance(symbols: List[str], period: str = "1y") -> str:
    """Compare performance of multiple stocks over a specified period (1y, 6m, 3m, 1m)."""
    if len(symbols) > 5:
        return "❌ Please limit comparison to 5 stocks maximum"

    try:
        performance_data = {}

        for symbol in symbols:
            stock = yf.Ticker(symbol)
            hist = stock.history(period=period)
            if not hist.empty:
                start_price = hist["Close"].iloc[0]
                end_price = hist["Close"].iloc[-1]
                performance = ((end_price - start_price) / start_price) * 100
                performance_data[symbol] = performance

        result = f"📈 Stock Performance Comparison ({period}):\n"
        sorted_stocks = sorted(
            performance_data.items(), key=lambda x: x[1], reverse=True
        )

        for stock, performance in sorted_stocks:
            result += f"• {stock}: {performance:+.2f}%\n"

        return result

    except Exception as e:
        return f"❌ Error comparing stocks: {str(e)}"


# Create the Financial Analysis Agent
financial_analysis_agent = Agent(
    model=bedrock_model,  # Using the same bedrock_model from Step 1
    system_prompt=FINANCIAL_ANALYSIS_PROMPT,
    tools=[get_stock_analysis, create_diversified_portfolio, compare_stock_performance],
    callback_handler=None,
)

@app.entrypoint
async def invoke(payload):
    """Your AI agent function"""
    print(payload)
    user_message = payload["prompt"]
    async for event in financial_analysis_agent.stream_async(user_message):
        print(event)
        if "data" in event:
            # Only stream text chunks to the client
            print(event["data"])
            yield event["data"]


if __name__ == "__main__":
    app.run()

In [ ]:
# Configure AgentCore Runtime deployment settings
agentcore_runtime = Runtime()

agent_name = "customer_support_agent"

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="strands_agentcore_SageMaker_deploy.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements-agentcore.txt",
    region=region,
    agent_name=agent_name,
    authorizer_configuration=auth_config,
    disable_otel=True, # Required to capture traces in SageMaker managed MLflow
)

print("Configuration completed ✓")

### Launching agent to AgentCore Runtime

Now that we have a Dockerfile, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime.

In [ ]:
# Deploy agent to AgentCore Runtime (creates ECR repo and runtime)
print("Launching Agent server to AgentCore Runtime...")
print("This may take several minutes...")

launch_result = agentcore_runtime.launch(
    env_vars={"OTEL_PYTHON_EXCLUDED_URLS": "/ping,/invocations"}
)

print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

### Add SageMaker MLflow IAM permissions to the AgentCore agent IAM role 
We add Sagemaker-mlflow IAM permission to the AgentCore runtime IAM role, to allow the AgentCore runtime to interact with Sagemaker managed MLflow and log traces. 

See AWS documentation for IAM actions supported for [Sagemaker managed MLflow](https://docs.aws.amazon.com/sagemaker/latest/dg/mlflow-create-tracking-server-iam.html#mlflow-create-tracking-server-iam-actions)


In [ ]:
# Retrieve the IAM Role attached to the AgentCore runtime
agent_status = agentcore_runtime.status()
print(agent_status)
execution_role_arn = agent_status.config.execution_role
print(f"Agent execution role arn: {execution_role_arn}")

In [ ]:
# Import IAM permissions module
import boto3
import json
from utils.add_iam_permissions import add_sagemaker_mlflow_s3_permissions

In [ ]:
# update IAM role permissions on the AgentCore runtime
print("\nAdding SageMaker MLflow")
add_sagemaker_mlflow_s3_permissions(execution_role_arn)
print("✓ IAM permissions updated successfully")

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

In [ ]:
# Authenticate user and get bearer token for API access
bearer_token = reauthenticate_user(client_id=cognito_config["client_id"])

In [ ]:
def invoke_endpoint(
    agent_arn: str,
    payload,
    session_id: str,
    bearer_token: Optional[str],
    region: str = "us-east-1",
    endpoint_name: str = "DEFAULT",
) -> Any:
    """Invoke agent endpoint using HTTP request with bearer token."""
    escaped_arn = urllib.parse.quote(agent_arn, safe="")
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_arn}/invocations"
    headers = {
        "Authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json",
        "X-Amzn-Bedrock-AgentCore-Runtime-Session-Id": session_id,
    }

    try:
        body = json.loads(payload) if isinstance(payload, str) else payload
    except json.JSONDecodeError:
        body = {"payload": payload}

    try:
        response = requests.post(
            url,
            params={"qualifier": endpoint_name},
            headers=headers,
            json=body,
            timeout=100,
            stream=True,
        )
        last_data = False
        for line in response.iter_lines(chunk_size=1):
            if line:
                line = line.decode("utf-8")
                if line.startswith("data: "):
                    last_data = True
                    line = line[6:]
                    line = line.replace('"', "")
                    yield line
                elif line:
                    line = line.replace('"', "")
                    if last_data:
                        yield "\n" + line
                    last_data = False

    except requests.exceptions.RequestException as e:
        print("Failed to invoke agent endpoint: %s", str(e))
        raise

In [ ]:
for chunk in invoke_endpoint(
    agent_arn=launch_result.agent_arn,
    payload={
        "prompt": "I make $100,000/month and want to start investing into moderate risk portfolio and also analyze stocks. Help me create an investment portfolio for me. Provide a deep analysis of Top 5 stocks in NYSE. Furthermore, recommend which is the of the top stock is high risk for my portfolio."
    },
    session_id=str(uuid.uuid4()),
    bearer_token=bearer_token,
):
    print(chunk.replace("\\n", "\n"), end="")


## Outputs in SageMaker managed MLflow
See the AgentCore runtime traced in the SageMaker managed MLflow tracking server. 

- Go to the SageMaker managed MLflow tracking server and open the `traces` tab for the MLflow experiment.
- If you used the default values in this notebook, your MLflow experiment will `_SAGEMAKER_MLFLOW_EXPERIMENT_NAME = "customer_support_genai_agentcore"`
- Analyze the mlflow traces by clicking through various options: `Trace breakdown`, `Show execution timeline`, `Inputs / Outputs`, `Attributes`, `Events`. 
- Notice and track the usage details of your agent: `gen_ai.usage.total_tokens`, `system_prompt`, `gen_ai.request.model`. 



## Cleanup (Optional)

Let's now clean up the AgentCore Runtime resources created.

In [ ]:
# Optional: Clean up Cognito user pool (commented out)
# delete_cognito_user_pool()

In [ ]:
# Get deployment details for cleanup (commented out)
# launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split("/")[1]

In [ ]:
# Optional: Delete AgentCore Runtime and ECR repository (commented out)
# agentcore_control_client = boto3.client("bedrock-agentcore-control", region_name=region)
# ecr_client = boto3.client("ecr", region_name=region)

# runtime_delete_response = agentcore_control_client.delete_agent_runtime(
#     agentRuntimeId=launch_result.agent_id,
# )

# response = ecr_client.delete_repository(
#     repositoryName=launch_result.ecr_uri.split("/")[1], force=True
# )